# Fase 3: Extracción de Detalles de Actores con Selenium - Parte final

Extraer información detallada de los 10 principales actores de cada una de las películas extraidas en la fase 1 utilizando Selenium (desde la página e IMDB). Debereis solo tener en cuenta los 10 primeros actores de cada película o corto (en caso de que los haya)La información que deberás extraer es:

Nombre.

Año nacimiento.

¿Por qué es conocido?

¿Qué hace?

Premios.

NOTA: La información de la API deberá ser almacenada en una lista de tuplas. Cada tupla corresponderá a una película. 



In [14]:
import requests
import re

from selenium import webdriver  # Selenium es una herramienta para automatizar la interacción con navegadores web.
from webdriver_manager.chrome import ChromeDriverManager  # ChromeDriverManager gestiona la instalación del controlador de Chrome.
from selenium.webdriver.common.keys import Keys  # Keys es útil para simular eventos de teclado en Selenium.
from selenium.webdriver.support.ui import Select  # Select se utiliza para interactuar con elementos <select> en páginas web.
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import pandas as pd

from time import sleep

pd.set_option('display.max_columns', None)  # Establece una opción de Pandas para mostrar todas las columnas de un DataFrame.

In [15]:

def obtener_detalles_actores(lista_actores,lista_detalles_actores):

    # Crear un diccionario llamado "datos_actor" para almacenar la información de cada uno de los actores. Un diccionario por cada actor.
    datos_actor = {}

    url_actores=(f"https://www.imdb.com/")

    # Inicializa el navegador Chrome
    driver = webdriver.Chrome()

    # Navega a la página web "https://www.imdb.com/"
    driver.get(url_actores)

    driver.maximize_window()

    sleep(2)

    # Aceptar las cookies en la página web

    driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()

    sleep(3)

    #Vamos a recorrer la lista de actores

    for actor in lista_actores:

            # Ingresar el actor de la lista de actores y coger los datos en el campo de búsqueda y presionar Enter
            
            driver.find_element("css selector", "#suggestion-search").send_keys(actor, Keys.ENTER)

            sleep(5)

            # Agrego el valor actor a la clave nombre_actor
            
            datos_actor["nombre_actor"] = actor
            
            # pinchamos en el resultado de la busqueda que nos lleva a la info del actor

            driver.execute_script("window.scrollTo(0,400)")
            
            sleep (2)
            
            driver.find_element('xpath','//*[@id="__next"]/main/div[2]/div[3]/section/div/div[1]/section[2]/div[2]/ul/li[1]/div[2]/div/a').click()
            
            sleep (3)

            try:
                resultado_que_hace = driver.find_element('xpath','//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[2]/div/ul').text
                lista_que_hace = resultado_que_hace.split("\n")
                datos_actor["que_hace"] = lista_que_hace    
            
            except:                
                print("Error al encontrar la seccion 'que hace'")
                datos_actor["que_hace"] = "None"
            
            sleep(2) 

            try:            
                driver.execute_script("window.scrollTo(0,500)")
                anno_nacimiento = driver.find_element("xpath", '//*[@id="__next"]/main/div/section[1]/section/div[3]/section/section/div[3]/div[2]/div[2]/section/aside/div/span[2]').text
                resultado_anno_nacimiento = re.search(r'\b\d{4}\b', anno_nacimiento).group()
                datos_actor["anno_nacimiento"] = resultado_anno_nacimiento    
            except:
                print("Error al extrear el año de nacimiento")
                datos_actor['anno_nacimiento'] = "0"

            sleep(3) 
            i = 500
            while i<2000:
                try:
                    driver.execute_script(f"window.scrollTo(0,{i})")
                    sleep(3)
                    
                    try:
                        texto_premios = driver.find_element('xpath','//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[1]/div/ul/li/div/ul').text
                        break
                    except:
                        texto_premios = driver.find_element('xpath','//*[@id="__next"]/main/div/section[1]/div/section/div/div[1]/section[2]/div/ul/li/div/ul').text
                        break
                except:
                    i+=500
            
            try:
                cantidad_premios = re.search(r'(\d+)\s+premios?',texto_premios, re.IGNORECASE).group(1)
                datos_actor['premios'] = cantidad_premios
        
            except:
                print("Error al extrear los premios")
                datos_actor['premios'] = "0"

            driver.execute_script(f"window.scrollTo({i},0)") 
            
            sleep(4)
            listas_pelis_conocido_por = []
            for x in range(1,5):
                    
                try:
                    driver.execute_script("window.scrollTo(0,2000)")
                    sleep(3)
                    resultado_conocido_por = driver.find_element('css selector',f'#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > div.celwidget > section:nth-child(1) > div.sc-af50cbe8-0.fqoRPX > div > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div:nth-child({x}) > div.ipc-primary-image-list-card__content > div.ipc-primary-image-list-card__content-top').text
                    listas_pelis_conocido_por.append(resultado_conocido_por)
                except:                    
                    print("Error al encontrar las peliculas")                    
                    listas_pelis_conocido_por.append("None")

            sleep(2)

            datos_actor["peliculas"] = listas_pelis_conocido_por 
            
            #driver.back()

            try:
                tupla_actor = (datos_actor['nombre_actor'], int(datos_actor['anno_nacimiento']), datos_actor["peliculas"], datos_actor["que_hace"], int(datos_actor['premios']))
                print(type(tupla_actor))
            except:
                print("Error al crear la tupla. Creamos tupla con null")
                tupla_actor = (datos_actor['nombre_actor'], 0, ["None"], ["None"], 0)

            lista_detalles_actores.append(tupla_actor)
                        
            datos_actor = {}
        
    print("----------------")
    
    #print(lista_detalles_actores)

    return(lista_detalles_actores)

In [3]:

def obtener_actores_from_peliculas(lista_id_peliculas, lista_actores):

        url_actores=(f"https://www.imdb.com/")

        # Inicializa el navegador Chrome
        driver = webdriver.Chrome()

        # Navega a la página web "https://www.imdb.com/"
        driver.get(url_actores)

        driver.maximize_window()

        sleep(2)

        # Aceptar las cookies en la página web

        driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()

        sleep(2)

        for pelicula in lista_id_peliculas:
                print(pelicula)
                
                try:
                        driver.find_element("css selector", "#suggestion-search").send_keys(pelicula, Keys.ENTER)
                except:
                        print("Pelicula no encontrada")

                driver.execute_script("window.scrollTo(0,500)")
                
                sleep(2)

                for i in range(1,11):

                        try:
                                actor = driver.find_element('css selector',f'#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-c41b9732-0.NeSef > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.jgUBLM.title-cast.title-cast--movie.celwidget > div.ipc-shoveler.ipc-shoveler--base.ipc-shoveler--page0.title-cast__grid > div.ipc-sub-grid.ipc-sub-grid--page-span-2.ipc-sub-grid--wraps-at-above-l.ipc-shoveler__grid > div:nth-child({i})').text.split("\n")[0]                               
                                if actor not in lista_actores:
                                        lista_actores.append(actor)
                                        
                        except:
                                print(f"Error al encontrar los 10 primeros actores de la pelicula {pelicula}")
                sleep(2)                               
                driver.back()
                sleep(2)
                
        return lista_actores

In [26]:
# Importamos los datos obtenidos de la Fase1 y los guardamos en una variable con formato DataFrame
csv_url = "https://raw.githubusercontent.com/fernandaMarti/Proyecto-da-promo-H-modulo-2-team1-cine_freaks/main/Fase1.csv"
df= pd.read_csv(csv_url)

# Sacamos un lista con los datos de la columna 'id_pelicula'
lista_id_peliculas = list(df['id_pelicula'])

In [18]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   tipo_pelicula    1000 non-null   object
 1   titulo_pelicula  1000 non-null   object
 2   anno_estreno     1000 non-null   int64 
 3   mes_estreno      924 non-null    object
 4   id_pelicula      1000 non-null   object
 5   genero_pelicula  1000 non-null   object
dtypes: int64(1), object(5)
memory usage: 47.0+ KB


In [28]:
#obtener lista de actores usando una lista de id peliculas de un CSV
resultados_lista_actores = obtener_actores_from_peliculas(lista_id_peliculas, lista_actores) 

tt7488946
Error al encontrar los 10 primeros actores de la pelicula tt7488946
Error al encontrar los 10 primeros actores de la pelicula tt7488946
Error al encontrar los 10 primeros actores de la pelicula tt7488946
Error al encontrar los 10 primeros actores de la pelicula tt7488946
Error al encontrar los 10 primeros actores de la pelicula tt7488946
Error al encontrar los 10 primeros actores de la pelicula tt7488946
Error al encontrar los 10 primeros actores de la pelicula tt7488946
Error al encontrar los 10 primeros actores de la pelicula tt7488946
tt26626399
Error al encontrar los 10 primeros actores de la pelicula tt26626399
Error al encontrar los 10 primeros actores de la pelicula tt26626399
Error al encontrar los 10 primeros actores de la pelicula tt26626399
Error al encontrar los 10 primeros actores de la pelicula tt26626399
Error al encontrar los 10 primeros actores de la pelicula tt26626399
Error al encontrar los 10 primeros actores de la pelicula tt26626399
Error al encontrar lo

In [29]:
len(lista_actores)


2539

In [30]:
df_lista_actores_2 = pd.DataFrame(lista_actores)

In [31]:
df_lista_actores_2

,0
0,Dan Jopling
1,Croix Provence
2,Aswathi Balan
3,Vijay Karthik
4,Chivvaun Abigial
...,...
2534,Durukan Çelikkaya
2535,Tuana Almaci
2536,Keci Memos
2537,Baris Bayram


In [32]:
df_lista_actores_2.to_csv("lista_actores_2.csv", index=False)

In [19]:
import csv
with open('actores.csv', 'w', newline='') as file:
    
    writer = csv.writer(file)
    writer.writerow(lista_actores)

In [31]:
# para que no se pierda la lista obtenida anteriormente:

lista_detalles_actores=[]
resultado_detalles_actores = obtener_detalles_actores(lista_actores,lista_detalles_actores)

Byom
Error al extrear los premios
Error al extrear el año de nacimiento
Rose LaRue
Error al extrear los premios
Error al extrear el año de nacimiento
Error al encontrar las peliculas
Error al encontrar las peliculas
Error al encontrar las peliculas
Error al encontrar las peliculas
Error al encontrar la seccion 'que hace'
Aaron Jackson
Error al extrear los premios
Error al extrear el año de nacimiento
Becky Brown
Error al extrear los premios
Error al extrear el año de nacimiento
Cassidy Reyes
Error al extrear los premios
Error al extrear el año de nacimiento
Mercedes Gutierrez
Error al extrear los premios
Error al extrear el año de nacimiento
Tay Lindsey
Error al extrear los premios
Error al extrear el año de nacimiento
Deborah Smith Ford
Error al crear la tupla. Creamos tupla con null
Jason Menz
Error al extrear los premios
Error al extrear el año de nacimiento
Error al encontrar las peliculas
Error al encontrar las peliculas
Error al encontrar las peliculas
Error al encontrar las peli

ElementClickInterceptedException: Message: element click intercepted: Element is not clickable at point (454, 581)
  (Session info: chrome=124.0.6367.119)
Stacktrace:
	GetHandleVerifier [0x00007FF790E61572+60802]
	(No symbol) [0x00007FF790DDAC72]
	(No symbol) [0x00007FF790C97CE4]
	(No symbol) [0x00007FF790CEE877]
	(No symbol) [0x00007FF790CEC344]
	(No symbol) [0x00007FF790CE980A]
	(No symbol) [0x00007FF790CE89E2]
	(No symbol) [0x00007FF790CDAE8B]
	(No symbol) [0x00007FF790D0AB7A]
	(No symbol) [0x00007FF790CDA7C6]
	(No symbol) [0x00007FF790D0AD90]
	(No symbol) [0x00007FF790D2A224]
	(No symbol) [0x00007FF790D0A923]
	(No symbol) [0x00007FF790CD8FEC]
	(No symbol) [0x00007FF790CD9C21]
	GetHandleVerifier [0x00007FF79116420D+3217949]
	GetHandleVerifier [0x00007FF7911A61A7+3488183]
	GetHandleVerifier [0x00007FF79119F12F+3459391]
	GetHandleVerifier [0x00007FF790F1B936+823622]
	(No symbol) [0x00007FF790DE600F]
	(No symbol) [0x00007FF790DE0F34]
	(No symbol) [0x00007FF790DE10C2]
	(No symbol) [0x00

In [32]:
import csv
with open('detalles_actores.csv', 'w', newline='') as file:
    
    writer = csv.writer(file)
    writer.writerow(lista_detalles_actores)

In [ ]:
print(len(lista_actores))
df = pd.DataFrame(lista_detalles_actores)
df

In [8]:
lista_actores = []
lista_id_peliculas = [ 'tt1929361','tt29009897','tt4209582','tt8042420','tt4209582']

In [9]:
# Llamamos a la funcion para obtener los nombres de los actores
resultados_lista_actores = obtener_actores_from_peliculas(lista_id_peliculas, lista_actores) 

tt1929361
tt29009897
tt4209582
tt8042420
Error al encontrar los 10 primeros actores de la pelicula tt8042420
Error al encontrar los 10 primeros actores de la pelicula tt8042420
Error al encontrar los 10 primeros actores de la pelicula tt8042420
Error al encontrar los 10 primeros actores de la pelicula tt8042420
Error al encontrar los 10 primeros actores de la pelicula tt8042420
Error al encontrar los 10 primeros actores de la pelicula tt8042420
tt4209582


In [16]:
lista_actores = ['AdamSandler','Leonardo DiCaprio','Nicole Kidman']
lista_actores

['AdamSandler', 'Leonardo DiCaprio', 'Nicole Kidman']

In [ ]:
Adam Sandler, , Leonardo Dicaprio

In [17]:
lista_detalles_actores=[]

In [ ]:
# Llamamos a la funcion para obtener los detalles de los actores
resultado_detalles_actores = obtener_detalles_actores(lista_actores,lista_detalles_actores)

In [132]:
lista_detalles_actores

[('Sam Richardson',
  1984.0,
  ['Veep', 'Detroiters', 'Mike y Dave buscan rollo serio', 'Somos los Miller'],
  ['Reparto', 'Guion', 'Producción'],
  3.0),
 ('Kirk Caouette',
  'NaN',
  ['X-Men 2', 'Dream with You', 'A Xiu Luo', 'Elektra'],
  ['Especialistas', 'Guion', 'Dirección'],
  4.0),
 ('Jan Vondrácek',
  1966.0,
  ['Jan Palach', 'Ceské století', 'Matematika zlocinu', 'Teroristka'],
  ['Reparto'],
  'NaN')]

In [101]:
lista_detalles_actores = []


In [110]:
df2.iloc[765]['peliculas']


"Jan Palach', 'Ceské století', 'Matematika zlocinu', 'Teroristka'"

In [111]:
list1 = ['Jan Palach', 'Ceské století', 'Matematika zlocinu', 'Teroristka']
list1

['Jan Palach', 'Ceské století', 'Matematika zlocinu', 'Teroristka']

In [112]:
df2.iloc[765]['que_hace']

"Reparto'"

In [113]:
list2 =['Reparto' ]
list2

['Reparto']

In [114]:
tup = (df2.iloc[765]['nombre_actor'] , df2.iloc[765]['anno_nacimiento'],list1,list2,df2.iloc[765]['premios'])

In [115]:

tup



('Jan Vondrácek',
 1966.0,
 ['Jan Palach', 'Ceské století', 'Matematika zlocinu', 'Teroristka'],
 ['Reparto'],
 nan)

In [116]:
lista_detalles_actores.append(tup)
lista_detalles_actores

[('Sam Richardson',
  1984.0,
  ['Veep', 'Detroiters', 'Mike y Dave buscan rollo serio', 'Somos los Miller'],
  ['Reparto', 'Guion', 'Producción'],
  3.0),
 ('Kirk Caouette',
  nan,
  ['X-Men 2', 'Dream with You', 'A Xiu Luo', 'Elektra'],
  ['Especialistas', 'Guion', 'Dirección'],
  4.0),
 ('Jan Vondrácek',
  1966.0,
  ['Jan Palach', 'Ceské století', 'Matematika zlocinu', 'Teroristka'],
  ['Reparto'],
  nan)]

In [130]:
lista_detalles_actores = [('Sam Richardson', 1984.0, ['Veep', 'Detroiters', 'Mike y Dave buscan rollo serio', 'Somos los Miller'],['Reparto', 'Guion', 'Producción'], 3.0),
 ('Kirk Caouette','NaN',['X-Men 2', 'Dream with You', 'A Xiu Luo', 'Elektra'],['Especialistas', 'Guion', 'Dirección'],4.0),
 ('Jan Vondrácek',1966.0,['Jan Palach', 'Ceské století', 'Matematika zlocinu', 'Teroristka'],['Reparto'],'NaN')]

In [131]:
lista_detalles_actores

[('Sam Richardson',
  1984.0,
  ['Veep', 'Detroiters', 'Mike y Dave buscan rollo serio', 'Somos los Miller'],
  ['Reparto', 'Guion', 'Producción'],
  3.0),
 ('Kirk Caouette',
  'NaN',
  ['X-Men 2', 'Dream with You', 'A Xiu Luo', 'Elektra'],
  ['Especialistas', 'Guion', 'Dirección'],
  4.0),
 ('Jan Vondrácek',
  1966.0,
  ['Jan Palach', 'Ceské století', 'Matematika zlocinu', 'Teroristka'],
  ['Reparto'],
  'NaN')]

In [ ]:
# Convertimos el DataFrame a formato .csv
csv_save_url = "https://raw.githubusercontent.com/fernandaMarti/Proyecto-da-promo-H-modulo-2-team1-cine_freaks/main/Fase3.csv"

lista_detalles_actores.to_csv(csv_save_url, index=False)